### Домашнее задание к уроку 6. Задача lookalike (Positive Unlabeled Learning)

1. Взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php).
2. Обучить любой классификатор (какой вам нравится).
3. Разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть.
4. Применить random negative sampling для построения классификатора в новых условиях.
5. Сравнить качество с решением из пункта 3 (построить отчет - таблицу метрик).

Датасет - https://archive.ics.uci.edu/ml/datasets/Maternal+Health+Risk+Data+Set

Attribute Information:

1. Age: Any ages in years when a women during pregnant.
2. SystolicBP: Upper value of Blood Pressure in mmHg, another significant attribute during pregnancy.
3. DiastolicBP: Lower value of Blood Pressure in mmHg, another significant attribute during pregnancy.
4. BS: Blood glucose levels is in terms of a molar concentration, mmol/L.
5. HeartRate: A normal resting heart rate in beats per minute.
6. Risk Level: Predicted Risk Intensity Level during pregnancy considering the previous attribute.

In [1]:
import pandas as pd
import numpy as np

# словарь для записи результатов
models_results = {'Model': [], 'F-Score': [], 'Precision': [], 'Recall': []}

data = pd.read_csv('risk/Maternal_Health_Risk_Data_Set.csv')
data.head(3)

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
0,25,130,80,15.0,98.0,86,high risk
1,35,140,90,13.0,98.0,70,high risk
2,29,90,70,8.0,100.0,80,high risk


In [2]:
data['RiskLevel'].value_counts()

low risk     406
mid risk     336
high risk    272
Name: RiskLevel, dtype: int64

Целевая переменная "RiskLevel", принимает значение: low risk, mid risk и high risk.

Разделим на 2 класса: 
1 - mid risk и high risk, т.е. есть риск заболевания.
0 - low risk, низкий риск заболевания.

In [3]:
data['RiskLevel'] = data['RiskLevel'].apply(lambda x: 0 if x=='low risk' else 1, 0).astype('int')

In [4]:
data.head(3)

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
0,25,130,80,15.0,98.0,86,1
1,35,140,90,13.0,98.0,70,1
2,29,90,70,8.0,100.0,80,1


In [5]:
data.dtypes

Age              int64
SystolicBP       int64
DiastolicBP      int64
BS             float64
BodyTemp       float64
HeartRate        int64
RiskLevel        int32
dtype: object

У нас есть 6 признаков и 1 целевая переменная - нужно определить есть риск заболевания или нет.

Посмотрим на соотношение классов

In [6]:
data['RiskLevel'].value_counts()

1    608
0    406
Name: RiskLevel, dtype: int64

Разбиваем выборку на тренировочную и тестовую части и обучаем модель (в примере - градиентный бустинг)

In [7]:
from sklearn.model_selection import train_test_split

x_data = data.copy().drop(columns='RiskLevel', axis=1)
y_data = data['RiskLevel']

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

In [8]:
import xgboost as xgb

model = xgb.XGBClassifier()

model.fit(x_train, y_train)
y_predict = model.predict(x_test)

Проверяем качество

In [9]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

def evaluate_results(y_test, y_predict):
    f1 = f1_score(y_test, y_predict)
    rec = recall_score(y_test, y_predict, average='binary')
    prec = precision_score(y_test, y_predict, average='binary')
    # print(f'f1_score={f1*100:.2f}%, recall={rec*100:.2f}%, precision={prec*100:.2f}%')
    return f1, rec, prec

f1, rec, prec = evaluate_results(y_test, y_predict)

models_results['Model'].append('XGBClassifier')
models_results['F-Score'].append(round(f1, 4))
models_results['Precision'].append(round(prec, 4))
models_results['Recall'].append(round(rec, 4))

pd.DataFrame(data=models_results)

,Model,F-Score,Precision,Recall
0,XGBClassifier,0.9255,0.9077,0.944


### Теперь очередь за PU learning

Представим, что нам неизвестны негативы и часть позитивов

In [10]:
mod_data = x_train.copy()
mod_data['RiskLevel'] = y_train
mod_data = mod_data.reset_index(drop=True)

# получим индексы позитивов
pos_ind = np.where(mod_data.iloc[:, -1].values == 1)[0]

# перемешаем
np.random.shuffle(pos_ind)

# возьмем 50% от позитивныз примеров
perc = 0.5
pos_sample_len = int(np.ceil(perc * len(pos_ind)))

print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 242/483 as positives and unlabeling the rest


In [11]:
mod_data.head()

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
0,55,140,95,19.0,98.0,77,1
1,23,120,90,7.9,98.0,70,1
2,60,120,80,6.8,98.0,77,1
3,25,120,90,15.0,98.0,80,1
4,32,140,90,18.0,98.0,88,1


Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)

In [12]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    569
 1    242
Name: class_test, dtype: int64


In [13]:
mod_data.head(10)

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel,class_test
0,55,140,95,19.0,98.0,77,1,1
1,23,120,90,7.9,98.0,70,1,1
2,60,120,80,6.8,98.0,77,1,-1
3,25,120,90,15.0,98.0,80,1,-1
4,32,140,90,18.0,98.0,88,1,1
5,20,110,60,7.0,100.0,70,1,-1
6,30,120,80,9.0,101.0,76,1,-1
7,16,90,65,6.9,98.0,76,1,1
8,55,129,85,7.5,98.0,88,0,-1
9,22,90,60,6.8,98.0,77,0,-1


random negative sampling

Помним, что x_data содержит целевой признак, который будем использовать для оценки качества. Отделим [:-2] как истинный класс для проверки, и [:,-1] как данные для входной разметки PUL.

In [14]:
mod_data = mod_data.sample(frac=1)

data_N = mod_data[mod_data['class_test'] == -1]
data_P = mod_data[mod_data['class_test'] == 1]

neg_sample = data_N[:data_P.shape[0]]
sample_test = data_N[data_P.shape[0]:]
pos_sample = data_P.copy()

print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(242, 8) (242, 8)


In [15]:
sample_train.head()

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel,class_test
345,22,120,90,7.8,98.0,82,1,1
190,23,90,60,7.5,98.0,76,0,-1
607,40,140,100,18.0,98.0,90,1,1
671,55,140,95,19.0,98.0,77,1,1
14,28,90,60,7.5,98.0,82,1,-1


In [16]:
model = xgb.XGBClassifier()
sample_train.loc[sample_train['class_test'] == -1, 'class_test'] = 0

model.fit(sample_train.drop(columns=['class_test', 'RiskLevel']),
          sample_train['class_test'])

y_predict = model.predict(x_test)

f1, rec, prec = evaluate_results(y_test, y_predict)

models_results['Model'].append('Positive Unlabeled Learning')
models_results['F-Score'].append(round(f1, 4))
models_results['Precision'].append(round(prec, 4))
models_results['Recall'].append(round(rec, 4))

pd.DataFrame(data=models_results)

,Model,F-Score,Precision,Recall
0,XGBClassifier,0.9255,0.9077,0.944
1,Positive Unlabeled Learning,0.7788,0.8713,0.704
